In [7]:
import pandas as pd
import numpy as np

In [14]:
import matplotlib.pyplot as plt

In [8]:
h5 = pd.HDFStore("/home/pchatha/mimic/data/all_hourly_data.h5")

#### Narrow Down Codes

In [10]:
stay_lengths = interventions.groupby("subject_id").size()
stay_lengths = stay_lengths[(stay_lengths >= 16) & (stay_lengths <= 128)]
stay_lengths.name="stay_length"

In [62]:
codes = h5["codes"].reset_index()[["subject_id", "icd9_codes"]].drop_duplicates(["subject_id"])
codes = codes.set_index("subject_id").join(stay_lengths, how="right")
codes = codes.explode("icd9_codes")

In [63]:
code_counts = codes["icd9_codes"].value_counts()
code_counts = code_counts[code_counts >= 100]
code_counts.name = "count"
code_counts = code_counts.to_frame()

In [64]:
codes = codes.merge(code_counts, left_on="icd9_codes", right_index=True, how="left")
codes["icd9_codes"] = codes["icd9_codes"].mask(codes["count"].isna())
codes["icd9_codes"] = codes["icd9_codes"].fillna("unk")

In [66]:
codes.head()

,icd9_codes,stay_length,count
subject_id,,,
4,042,41,263.0
4,unk,41,NaN
4,7994,41,117.0
4,2763,41,364.0
4,7907,41,588.0


In [59]:
codes["icd9_codes"].mask(codes["count"].isna())

subject_id
4          042
4          NaN
4         7994
4         2763
4         7907
         ...  
99999      NaN
99999      NaN
99999     4019
99999    25000
99999     4139
Name: icd9_codes, Length: 319676, dtype: object

In [54]:
codes[codes["count"].notna()]["icd9_codes"].nunique()

515

In [53]:
codes.head()

,icd9_codes,stay_length,count
subject_id,,,
4,042,41,263.0
4,1363,41,NaN
4,7994,41,117.0
4,2763,41,364.0
4,7907,41,588.0
